<h1 style="text-align:center"> INFO 323: Cloud Computing and Big Data</h1>
<h2 style="text-align:center"> College of Computing and Informatics</h2>
<h2 style="text-align:center">Drexel University</h2>

<h3 style="text-align:center"> Spark Preprocessing</h3>
<h3 style="text-align:center"> Yuan An, PhD</h3>
<h3 style="text-align:center">Associate Professor</h3>

# Formatting Models According to Your Use Case
To preprocess data for Spark’s different advanced analytics tools, you must consider your end
objective. The following list walks through the requirements for input data structure for each
advanced analytics task in MLlib:
* In the case of most classification and regression algorithms, you want to get your data into **a column of type Double to represent the label and a column of type Vector (either dense or sparse) to represent the features**.
* In the case of recommendation, you want to get your data into a column of users, **a column of items (say movies or books), and a column of ratings**.
* In the case of unsupervised learning, **a column of type Vector (either dense or sparse)** is needed to represent the features.
* In the case of graph analytics, you will want **a DataFrame of vertices and a DataFrame of edges**.

The best way to get your data in these formats is through transformers. Transformers are functions that
accept a DataFrame as an argument and return a new DataFrame as a response.

## Read in Several Sample Datasets

In [0]:
sales = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("dbfs:/FileStore/tables/retail-data/by-day/*.csv")\
  .coalesce(5)\
  .where("Description IS NOT NULL")
    

In [0]:
sales.show(2)

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
 580538| 23084| RABBIT NIGHT LIGHT| 48|2011-12-05 08:38:00| 1.79| 14075.0|United Kingdom|
 580538| 23077|DOUGHNUT LIP GLOSS | 20|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows

In [0]:
sales.count()

Out[3]: 540455

In [0]:
fakeIntDF = spark.read.parquet("dbfs:/FileStore/tables/simple-ml-integers")
simpleDF = spark.read.json("dbfs:/FileStore/tables/simple-ml")
scaleDF = spark.read.parquet("dbfs:/FileStore/tables/simple-ml-scaling")

In [0]:
fakeIntDF.show(2)

+----+----+----+
int1|int2|int3|
+----+----+----+
 4| 5| 6|
 7| 8| 9|
+----+----+----+
only showing top 2 rows

In [0]:
simpleDF.show(2)

+-----+----+------+------------------+
color| lab|value1| value2|
+-----+----+------+------------------+
green|good| 1|14.386294994851129|
 blue| bad| 8|14.386294994851129|
+-----+----+------+------------------+
only showing top 2 rows

In [0]:
scaleDF.show(2)

+---+--------------+
 id| features|
+---+--------------+
 0|[1.0,0.1,-1.0]|
 1| [2.0,1.1,1.0]|
+---+--------------+
only showing top 2 rows

## Transformers

Transformers are functions that convert raw data in some way. This might be to create a new
interaction variable (from two other variables), to normalize a column, or to simply turn it into a
Double to be input into a model. Transformers are primarily used in preprocessing or feature
generation.

All transformers require you to specify, at a minimum, the inputCol and the outputCol, which
represent the column name of the input and output, respectively.

## High-Level Transformers
High-level transformers allow you to
concisely specify a number of transformations in one. 
These operate at a “high level”, and allow you
to avoid doing data manipulations or transformations one by one. In general, you should try to use the
highest level transformers you can, in order to minimize the risk of error and help you focus on the
business problem instead of the smaller details of implementation. While this is not always possible,
it’s a good objective.

### RFormula
The RFormula is the easiest transfomer to use when you have “conventionally” formatted data.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import RFormula

supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

+-----+----+------+------------------+--------------------+-----+
color| lab|value1| value2| features|label|
+-----+----+------+------------------+--------------------+-----+
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 16|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red|good| 45| 38.97187133755819|(10,[0,2,3,4,7],[...| 1.0|
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
+-----+----+------+------------------+--------------------+-----+
only showing top 20 rows

###  SQL Transformers
A SQLTransformer allows you to leverage Spark’s vast library of SQL-related manipulations just as
you would a MLlib transformation. Any SELECT statement you can use in SQL is a valid
transformation. The only thing you need to change is that instead of using the table name, you should
just use the keyword THIS. The following is a basic example of using SQLTransformer:

In [0]:
# COMMAND ----------

from pyspark.ml.feature import SQLTransformer

basicTransformation = SQLTransformer()\
  .setStatement("""
    SELECT sum(Quantity), count(*), CustomerID
    FROM __THIS__
    GROUP BY CustomerID
  """)

basicTransformation.transform(sales).show()

+-------------+--------+----------+
sum(Quantity)|count(1)|CustomerID|
+-------------+--------+----------+
 440| 143| 16916.0|
 854| 117| 17884.0|
 244| 31| 16561.0|
 491| 152| 13956.0|
 541| 27| 14285.0|
 119| 62| 14452.0|
 204| 76| 13533.0|
 630| 72| 17633.0|
 493| 64| 16629.0|
 34| 6| 14768.0|
 159| 38| 17267.0|
 1542| 30| 13094.0|
 274| 17| 13930.0|
 67| 9| 18277.0|
 8873| 80| 16656.0|
 123| 9| 13937.0|
 756| 67| 15145.0|
 67| 14| 13811.0|
 1232| 118| 14211.0|
 37720| 2491| 15311.0|
+-------------+--------+----------+
only showing top 20 rows

### VectorAssembler
The VectorAssembler is a tool you’ll use in nearly every single pipeline you generate. It helps
concatenate all your features into one big vector you can then pass into an estimator. It’s used
typically in the last step of a machine learning pipeline and takes as input a number of columns of
Boolean, Double, or Vector. This is particularly helpful if you’re going to perform a number of
manipulations using a variety of transformers and need to gather all of those results together.
The output from the following code snippet will make it clear how this works:

In [0]:
# COMMAND ----------

from pyspark.ml.feature import VectorAssembler
va = VectorAssembler().setInputCols(["int1", "int2", "int3"]).setOutputCol("features")
va.transform(fakeIntDF).show()

+----+----+----+-------------+
int1|int2|int3| features|
+----+----+----+-------------+
 4| 5| 6|[4.0,5.0,6.0]|
 7| 8| 9|[7.0,8.0,9.0]|
 1| 2| 3|[1.0,2.0,3.0]|
+----+----+----+-------------+

## Working with Continuous Features

There are two common transformers for continuous features. First, you can convert continuous
features into categorical features via a process called bucketing, or you can scale and normalize your
features according to several different requirements. These transformers will only work on Double
types, so make sure you’ve turned any other numerical values to Double:

In [0]:
# COMMAND ----------

contDF = spark.range(20).selectExpr("cast(id as double)")

In [0]:
contDF.show(2)

+---+
 id|
+---+
0.0|
1.0|
+---+
only showing top 2 rows

### Bucketing
The most straightforward approach to bucketing or binning is using the Bucketizer. This will split a
given continuous feature into the buckets of your designation. You specify how buckets should be
created via an array or list of Double values.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import Bucketizer
bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(contDF).show()

+----+-------------------------------+
 id|Bucketizer_b97ad251a557__output|
+----+-------------------------------+
 0.0| 0.0|
 1.0| 0.0|
 2.0| 0.0|
 3.0| 0.0|
 4.0| 0.0|
 5.0| 1.0|
 6.0| 1.0|
 7.0| 1.0|
 8.0| 1.0|
 9.0| 1.0|
10.0| 2.0|
11.0| 2.0|
12.0| 2.0|
13.0| 2.0|
14.0| 2.0|
15.0| 2.0|
16.0| 2.0|
17.0| 2.0|
18.0| 2.0|
19.0| 2.0|
+----+-------------------------------+

In addition to splitting based on hardcoded values, another option is to split based on percentiles in
our data. This is done with QuantileDiscretizer, which will bucket the values into user-specified
buckets with the splits being determined by approximate quantiles values.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import QuantileDiscretizer
bucketer = QuantileDiscretizer().setNumBuckets(5).setInputCol("id")

In [0]:
bucketer = QuantileDiscretizer(numBuckets=5, inputCol="id", outputCol="buckets")

In [0]:
fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

+----+-------+
 id|buckets|
+----+-------+
 0.0| 0.0|
 1.0| 0.0|
 2.0| 0.0|
 3.0| 1.0|
 4.0| 1.0|
 5.0| 1.0|
 6.0| 1.0|
 7.0| 2.0|
 8.0| 2.0|
 9.0| 2.0|
10.0| 2.0|
11.0| 3.0|
12.0| 3.0|
13.0| 3.0|
14.0| 3.0|
15.0| 4.0|
16.0| 4.0|
17.0| 4.0|
18.0| 4.0|
19.0| 4.0|
+----+-------+

## StandardScaler
The StandardScaler standardizes a set of features to have zero mean and a standard deviation of 1.
The flag withStd will scale the data to unit standard deviation while the flag withMean (false by
default) will center the data prior to scaling it.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import StandardScaler
sScaler = StandardScaler().setInputCol("features").setOutputCol("scaledFeatures")
sScaler.fit(scaleDF).transform(scaleDF).show(10, False)

+---+--------------+------------------------------------------------------------+
id |features |scaledFeatures |
+---+--------------+------------------------------------------------------------+
0 |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
1 |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968] |
0 |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
1 |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968] |
1 |[3.0,10.1,3.0]|[3.5856858280031805,2.3609991401715313,1.7928429140015902] |
+---+--------------+------------------------------------------------------------+

### MinMaxScaler
The MinMaxScaler will scale the values in a vector (component wise) to the proportional values on
a scale from a given min value to a max value. If you specify the minimum value to be 0 and the
maximum value to be 1, then all the values will fall in between 0 and 1:

In [0]:
# COMMAND ----------

from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")

In [0]:
minMax = MinMaxScaler(inputCol="features")

In [0]:
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

+---+--------------+---------------------------------+
 id| features|MinMaxScaler_f28da468fbf5__output|
+---+--------------+---------------------------------+
 0|[1.0,0.1,-1.0]| [5.0,5.0,5.0]|
 1| [2.0,1.1,1.0]| [7.5,5.5,7.5]|
 0|[1.0,0.1,-1.0]| [5.0,5.0,5.0]|
 1| [2.0,1.1,1.0]| [7.5,5.5,7.5]|
 1|[3.0,10.1,3.0]| [10.0,10.0,10.0]|
+---+--------------+---------------------------------+

In [0]:
scaleDF.show()

+---+--------------+
 id| features|
+---+--------------+
 0|[1.0,0.1,-1.0]|
 1| [2.0,1.1,1.0]|
 0|[1.0,0.1,-1.0]|
 1| [2.0,1.1,1.0]|
 1|[3.0,10.1,3.0]|
+---+--------------+

### MaxAbsScaler
Rescale each feature individually to range [-1, 1] by dividing through the largest maximum absolute value in each feature. It does not shift/center the data, and thus does not destroy any sparsity.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import MaxAbsScaler
maScaler = MaxAbsScaler().setInputCol("features")
fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show(10, False)

+---+--------------+-------------------------------------------------------------+
id |features |MaxAbsScaler_6df7fd176eb9__output |
+---+--------------+-------------------------------------------------------------+
0 |[1.0,0.1,-1.0]|[0.3333333333333333,0.009900990099009903,-0.3333333333333333]|
1 |[2.0,1.1,1.0] |[0.6666666666666666,0.10891089108910892,0.3333333333333333] |
0 |[1.0,0.1,-1.0]|[0.3333333333333333,0.009900990099009903,-0.3333333333333333]|
1 |[2.0,1.1,1.0] |[0.6666666666666666,0.10891089108910892,0.3333333333333333] |
1 |[3.0,10.1,3.0]|[1.0,1.0,1.0] |
+---+--------------+-------------------------------------------------------------+

### ElementwiseProduct
Outputs the Hadamard product (i.e., the element-wise product) of each input vector with a provided “weight” vector. In other words, it scales each column of the dataset by a scalar multiplier.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors
scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
scalingUp = ElementwiseProduct()\
  .setScalingVec(scaleUpVec)\
  .setInputCol("features")
scalingUp.transform(scaleDF).show()

+---+--------------+---------------------------------------+
 id| features|ElementwiseProduct_410abad09764__output|
+---+--------------+---------------------------------------+
 0|[1.0,0.1,-1.0]| [10.0,1.5,-20.0]|
 1| [2.0,1.1,1.0]| [20.0,16.5,20.0]|
 0|[1.0,0.1,-1.0]| [10.0,1.5,-20.0]|
 1| [2.0,1.1,1.0]| [20.0,16.5,20.0]|
 1|[3.0,10.1,3.0]| [30.0,151.5,60.0]|
+---+--------------+---------------------------------------+

### Normalizer
Normalize a vector to have unit norm using the given p-norm.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import Normalizer
manhattanDistance = Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show(10, False)

+---+--------------+---------------------------------------------------------------+
id |features |Normalizer_456eca77429f__output |
+---+--------------+---------------------------------------------------------------+
0 |[1.0,0.1,-1.0]|[0.47619047619047616,0.047619047619047616,-0.47619047619047616]|
1 |[2.0,1.1,1.0] |[0.48780487804878053,0.26829268292682934,0.24390243902439027] |
0 |[1.0,0.1,-1.0]|[0.47619047619047616,0.047619047619047616,-0.47619047619047616]|
1 |[2.0,1.1,1.0] |[0.48780487804878053,0.26829268292682934,0.24390243902439027] |
1 |[3.0,10.1,3.0]|[0.18633540372670807,0.6273291925465838,0.18633540372670807] |
+---+--------------+---------------------------------------------------------------+

## Working with Categorical Features
The most common task for categorical features is indexing. Indexing converts a categorical variable
in a column to a numerical one that you can plug into machine learning algorithms.

### StringIndexer
The simplest way to index is via the StringIndexer, which maps strings to different numerical IDs.
Spark’s StringIndexer also creates metadata attached to the DataFrame that specify what inputs
correspond to what outputs. This allows us later to get inputs back from their respective index values:

In [0]:
# COMMAND ----------

from pyspark.ml.feature import StringIndexer
lblIndxr = StringIndexer().setInputCol("lab").setOutputCol("labelInd")
idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.show()

+-----+----+------+------------------+--------+
color| lab|value1| value2|labelInd|
+-----+----+------+------------------+--------+
green|good| 1|14.386294994851129| 1.0|
 blue| bad| 8|14.386294994851129| 0.0|
 blue| bad| 12|14.386294994851129| 0.0|
green|good| 15| 38.97187133755819| 1.0|
green|good| 12|14.386294994851129| 1.0|
green| bad| 16|14.386294994851129| 0.0|
 red|good| 35|14.386294994851129| 1.0|
 red| bad| 1| 38.97187133755819| 0.0|
 red| bad| 2|14.386294994851129| 0.0|
 red| bad| 16|14.386294994851129| 0.0|
 red|good| 45| 38.97187133755819| 1.0|
green|good| 1|14.386294994851129| 1.0|
 blue| bad| 8|14.386294994851129| 0.0|
 blue| bad| 12|14.386294994851129| 0.0|
green|good| 15| 38.97187133755819| 1.0|
green|good| 12|14.386294994851129| 1.0|
green| bad| 16|14.386294994851129| 0.0|
 red|good| 35|14.386294994851129| 1.0|
 red| bad| 1| 38.97187133755819| 0.0|
 red| bad| 2|14.386294994851129| 0.0|
+-----+----+------+------------------+--------+
only showing top 20 rows

We can also apply StringIndexer to columns that are not strings, in which case, they will be
converted to strings before being indexed:

In [0]:
# COMMAND ----------

valIndexer = StringIndexer().setInputCol("value1").setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show()

+-----+----+------+------------------+--------+
color| lab|value1| value2|valueInd|
+-----+----+------+------------------+--------+
green|good| 1|14.386294994851129| 0.0|
 blue| bad| 8|14.386294994851129| 7.0|
 blue| bad| 12|14.386294994851129| 1.0|
green|good| 15| 38.97187133755819| 3.0|
green|good| 12|14.386294994851129| 1.0|
green| bad| 16|14.386294994851129| 2.0|
 red|good| 35|14.386294994851129| 5.0|
 red| bad| 1| 38.97187133755819| 0.0|
 red| bad| 2|14.386294994851129| 4.0|
 red| bad| 16|14.386294994851129| 2.0|
 red|good| 45| 38.97187133755819| 6.0|
green|good| 1|14.386294994851129| 0.0|
 blue| bad| 8|14.386294994851129| 7.0|
 blue| bad| 12|14.386294994851129| 1.0|
green|good| 15| 38.97187133755819| 3.0|
green|good| 12|14.386294994851129| 1.0|
green| bad| 16|14.386294994851129| 2.0|
 red|good| 35|14.386294994851129| 5.0|
 red| bad| 1| 38.97187133755819| 0.0|
 red| bad| 2|14.386294994851129| 4.0|
+-----+----+------+------------------+--------+
only showing top 20 rows

### Converting Indexed Values Back to Text

In [0]:
# COMMAND ----------

from pyspark.ml.feature import IndexToString
labelReverse = IndexToString().setInputCol("labelInd")
labelReverse.transform(idxRes).show()

+-----+----+------+------------------+--------+----------------------------------+
color| lab|value1| value2|labelInd|IndexToString_6f3a2fd9ce15__output|
+-----+----+------+------------------+--------+----------------------------------+
green|good| 1|14.386294994851129| 1.0| good|
 blue| bad| 8|14.386294994851129| 0.0| bad|
 blue| bad| 12|14.386294994851129| 0.0| bad|
green|good| 15| 38.97187133755819| 1.0| good|
green|good| 12|14.386294994851129| 1.0| good|
green| bad| 16|14.386294994851129| 0.0| bad|
 red|good| 35|14.386294994851129| 1.0| good|
 red| bad| 1| 38.97187133755819| 0.0| bad|
 red| bad| 2|14.386294994851129| 0.0| bad|
 red| bad| 16|14.386294994851129| 0.0| bad|
 red|good| 45| 38.97187133755819| 1.0| good|
green|good| 1|14.386294994851129| 1.0| good|
 blue| bad| 8|14.386294994851129| 0.0| bad|
 blue| bad| 12|14.386294994851129| 0.0| bad|
green|good| 15| 38.97187133755819| 1.0| good|
green|good| 12|14.386294994851129| 1.0| good|
green| bad| 16|14.386294994851129| 0.0| bad|
 red|good| 35|14.386294994851129| 1.0| good|
 red| bad| 1| 38.97187133755819| 0.0| bad|
 red| bad| 2|14.386294994851129| 0.0| bad|
+-----+----+------+------------------+--------+----------------------------------+
only showing top 20 rows

### Indexing in Vectors
VectorIndexer is a helpful tool for working with categorical variables that are already found inside
of vectors in your dataset. This tool will automatically find categorical features inside of your input
vectors and convert them to categorical features with zero-based category indices.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors
idxIn = spark.createDataFrame([
  (Vectors.dense(1, 2, 3),1),
  (Vectors.dense(2, 5, 6),2),
  (Vectors.dense(1, 8, 9),3)
]).toDF("features", "label")
indxr = VectorIndexer()\
  .setInputCol("features")\
  .setOutputCol("idxed")\
  .setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show()

+-------------+-----+-------------+
 features|label| idxed|
+-------------+-----+-------------+
[1.0,2.0,3.0]| 1|[0.0,2.0,3.0]|
[2.0,5.0,6.0]| 2|[1.0,5.0,6.0]|
[1.0,8.0,9.0]| 3|[0.0,8.0,9.0]|
+-------------+-----+-------------+

### One-Hot Encoding
Indexing categorical variables is only half of the story. One-hot encoding is an extremely common
data transformation performed after indexing categorical variables.

In [0]:
simpleDF.show(2)

+-----+----+------+------------------+
color| lab|value1| value2|
+-----+----+------+------------------+
green|good| 1|14.386294994851129|
 blue| bad| 8|14.386294994851129|
+-----+----+------+------------------+
only showing top 2 rows

In [0]:
# COMMAND ----------

from pyspark.ml.feature import OneHotEncoder, StringIndexer
lblIndxr = StringIndexer().setInputCol("color").setOutputCol("colorInd")
colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
ohe = OneHotEncoder().setInputCol("colorInd")
ohe.fit(colorLab).transform(colorLab).show()

+-----+--------+----------------------------------+
color|colorInd|OneHotEncoder_744c0d0b9fb2__output|
+-----+--------+----------------------------------+
green| 1.0| (2,[1],[1.0])|
 blue| 2.0| (2,[],[])|
 blue| 2.0| (2,[],[])|
green| 1.0| (2,[1],[1.0])|
green| 1.0| (2,[1],[1.0])|
green| 1.0| (2,[1],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
green| 1.0| (2,[1],[1.0])|
 blue| 2.0| (2,[],[])|
 blue| 2.0| (2,[],[])|
green| 1.0| (2,[1],[1.0])|
green| 1.0| (2,[1],[1.0])|
green| 1.0| (2,[1],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
 red| 0.0| (2,[0],[1.0])|
+-----+--------+----------------------------------+
only showing top 20 rows

## Text Data Transformers

### Tokenizing Text
Tokenization is the process of converting free-form text into a list of “tokens” or individual words.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(20, False)

+-----------------------------------+------------------------------------------+
Description |DescOut |
+-----------------------------------+------------------------------------------+
RABBIT NIGHT LIGHT |[rabbit, night, light] |
DOUGHNUT LIP GLOSS |[doughnut, lip, gloss] |
12 MESSAGE CARDS WITH ENVELOPES |[12, message, cards, with, envelopes] |
BLUE HARMONICA IN BOX |[blue, harmonica, in, box] |
GUMBALL COAT RACK |[gumball, coat, rack] |
SKULLS WATER TRANSFER TATTOOS |[skulls, , water, transfer, tattoos] |
FELTCRAFT GIRL AMELIE KIT |[feltcraft, girl, amelie, kit] |
CAMOUFLAGE LED TORCH |[camouflage, led, torch] |
WHITE SKULL HOT WATER BOTTLE |[white, skull, hot, water, bottle] |
ENGLISH ROSE HOT WATER BOTTLE |[english, rose, hot, water, bottle] |
HOT WATER BOTTLE KEEP CALM |[hot, water, bottle, keep, calm] |
SCOTTIE DOG HOT WATER BOTTLE |[scottie, dog, hot, water, bottle] |
ROSE CARAVAN DOORSTOP |[rose, caravan, doorstop] |
GINGHAM HEART DOORSTOP RED |[gingham, heart, , doorstop, red] |
STORAGE TIN VINTAGE LEAF |[storage, tin, vintage, leaf] |
SET OF 4 KNICK KNACK TINS POPPIES |[set, of, 4, knick, knack, tins, poppies] |
POPCORN HOLDER |[popcorn, holder] |
GROW A FLYTRAP OR SUNFLOWER IN TIN |[grow, a, flytrap, or, sunflower, in, tin]|
AIRLINE BAG VINTAGE WORLD CHAMPION |[airline, bag, vintage, world, champion] |
AIRLINE BAG VINTAGE JET SET BROWN |[airline, bag, vintage, jet, set, brown] |
+-----------------------------------+------------------------------------------+
only showing top 20 rows

We can also create a Tokenizer that is not just based white space but a regular expression with the
RegexTokenizer. The format of the regular expression should conform to the Java Regular
Expression (RegEx) syntax:

In [0]:
# COMMAND ----------

from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
  .setInputCol("Description")\
  .setOutputCol("DescOut")\
  .setPattern(" ")\
  .setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------------------------------+
Description |DescOut |
+-----------------------------------+------------------------------------------+
RABBIT NIGHT LIGHT |[rabbit, night, light] |
DOUGHNUT LIP GLOSS |[doughnut, lip, gloss] |
12 MESSAGE CARDS WITH ENVELOPES |[12, message, cards, with, envelopes] |
BLUE HARMONICA IN BOX |[blue, harmonica, in, box] |
GUMBALL COAT RACK |[gumball, coat, rack] |
SKULLS WATER TRANSFER TATTOOS |[skulls, water, transfer, tattoos] |
FELTCRAFT GIRL AMELIE KIT |[feltcraft, girl, amelie, kit] |
CAMOUFLAGE LED TORCH |[camouflage, led, torch] |
WHITE SKULL HOT WATER BOTTLE |[white, skull, hot, water, bottle] |
ENGLISH ROSE HOT WATER BOTTLE |[english, rose, hot, water, bottle] |
HOT WATER BOTTLE KEEP CALM |[hot, water, bottle, keep, calm] |
SCOTTIE DOG HOT WATER BOTTLE |[scottie, dog, hot, water, bottle] |
ROSE CARAVAN DOORSTOP |[rose, caravan, doorstop] |
GINGHAM HEART DOORSTOP RED |[gingham, heart, doorstop, red] |
STORAGE TIN VINTAGE LEAF |[storage, tin, vintage, leaf] |
SET OF 4 KNICK KNACK TINS POPPIES |[set, of, 4, knick, knack, tins, poppies] |
POPCORN HOLDER |[popcorn, holder] |
GROW A FLYTRAP OR SUNFLOWER IN TIN |[grow, a, flytrap, or, sunflower, in, tin]|
AIRLINE BAG VINTAGE WORLD CHAMPION |[airline, bag, vintage, world, champion] |
AIRLINE BAG VINTAGE JET SET BROWN |[airline, bag, vintage, jet, set, brown] |
+-----------------------------------+------------------------------------------+
only showing top 20 rows

Another way of using the RegexTokenizer is to use it to output values matching the provided pattern
instead of using it as a gap.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
  .setInputCol("Description")\
  .setOutputCol("DescOut")\
  .setPattern(" ")\
  .setGaps(False)\
  .setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------+
Description |DescOut |
+-----------------------------------+------------------+
RABBIT NIGHT LIGHT |[ , ] |
DOUGHNUT LIP GLOSS |[ , , ] |
12 MESSAGE CARDS WITH ENVELOPES |[ , , , ] |
BLUE HARMONICA IN BOX |[ , , , ] |
GUMBALL COAT RACK |[ , ] |
SKULLS WATER TRANSFER TATTOOS |[ , , , , ] |
FELTCRAFT GIRL AMELIE KIT |[ , , ] |
CAMOUFLAGE LED TORCH |[ , ] |
WHITE SKULL HOT WATER BOTTLE |[ , , , , ] |
ENGLISH ROSE HOT WATER BOTTLE |[ , , , ] |
HOT WATER BOTTLE KEEP CALM |[ , , , ] |
SCOTTIE DOG HOT WATER BOTTLE |[ , , , ] |
ROSE CARAVAN DOORSTOP |[ , ] |
GINGHAM HEART DOORSTOP RED |[ , , , ] |
STORAGE TIN VINTAGE LEAF |[ , , ] |
SET OF 4 KNICK KNACK TINS POPPIES |[ , , , , , ]|
POPCORN HOLDER |[ ] |
GROW A FLYTRAP OR SUNFLOWER IN TIN |[ , , , , , ]|
AIRLINE BAG VINTAGE WORLD CHAMPION |[ , , , , ] |
AIRLINE BAG VINTAGE JET SET BROWN |[ , , , , ] |
+-----------------------------------+------------------+
only showing top 20 rows

### Removing Common Words

In [0]:
# COMMAND ----------

from pyspark.ml.feature import StopWordsRemover
englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover()\
  .setStopWords(englishStopWords)\
  .setInputCol("DescOut")
stops.transform(tokenized).show()

+--------------------+--------------------+-------------------------------------+
 Description| DescOut|StopWordsRemover_2d85f4e9edb2__output|
+--------------------+--------------------+-------------------------------------+
 RABBIT NIGHT LIGHT|[rabbit, night, l...| [rabbit, night, l...|
 DOUGHNUT LIP GLOSS |[doughnut, lip, g...| [doughnut, lip, g...|
12 MESSAGE CARDS ...|[12, message, car...| [12, message, car...|
BLUE HARMONICA IN...|[blue, harmonica,...| [blue, harmonica,...|
 GUMBALL COAT RACK|[gumball, coat, r...| [gumball, coat, r...|
SKULLS WATER TRA...|[skulls, , water,...| [skulls, , water,...|
FELTCRAFT GIRL AM...|[feltcraft, girl,...| [feltcraft, girl,...|
CAMOUFLAGE LED TORCH|[camouflage, led,...| [camouflage, led,...|
WHITE SKULL HOT W...|[white, skull, ho...| [white, skull, ho...|
ENGLISH ROSE HOT ...|[english, rose, h...| [english, rose, h...|
HOT WATER BOTTLE ...|[hot, water, bott...| [hot, water, bott...|
SCOTTIE DOG HOT W...|[scottie, dog, ho...| [scottie, dog, ho...|
ROSE CARAVAN DOOR...|[rose, caravan, d...| [rose, caravan, d...|
GINGHAM HEART DO...|[gingham, heart, ...| [gingham, heart, ...|
STORAGE TIN VINTA...|[storage, tin, vi...| [storage, tin, vi...|
SET OF 4 KNICK KN...|[set, of, 4, knic...| [set, 4, knick, k...|
 POPCORN HOLDER| [popcorn, holder]| [popcorn, holder]|
GROW A FLYTRAP OR...|[grow, a, flytrap...| [grow, flytrap, s...|
AIRLINE BAG VINTA...|[airline, bag, vi...| [airline, bag, vi...|
AIRLINE BAG VINTA...|[airline, bag, vi...| [airline, bag, vi...|
+--------------------+--------------------+-------------------------------------+
only showing top 20 rows

### Creating Word Combinations
With n-grams, we can look at sequences of words that commonly co-occur and use them as inputs to a
machine learning algorithm.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import NGram
unigram = NGram().setInputCol("DescOut").setN(1)
bigram = NGram().setInputCol("DescOut").setN(2)

In [0]:
unigram.transform(tokenized.select("DescOut")).show()

+--------------------+--------------------------+
 DescOut|NGram_248f72850d5b__output|
+--------------------+--------------------------+
[rabbit, night, l...| [rabbit, night, l...|
[doughnut, lip, g...| [doughnut, lip, g...|
[12, message, car...| [12, message, car...|
[blue, harmonica,...| [blue, harmonica,...|
[gumball, coat, r...| [gumball, coat, r...|
[skulls, , water,...| [skulls, , water,...|
[feltcraft, girl,...| [feltcraft, girl,...|
[camouflage, led,...| [camouflage, led,...|
[white, skull, ho...| [white, skull, ho...|
[english, rose, h...| [english, rose, h...|
[hot, water, bott...| [hot, water, bott...|
[scottie, dog, ho...| [scottie, dog, ho...|
[rose, caravan, d...| [rose, caravan, d...|
[gingham, heart, ...| [gingham, heart, ...|
[storage, tin, vi...| [storage, tin, vi...|
[set, of, 4, knic...| [set, of, 4, knic...|
 [popcorn, holder]| [popcorn, holder]|
[grow, a, flytrap...| [grow, a, flytrap...|
[airline, bag, vi...| [airline, bag, vi...|
[airline, bag, vi...| [airline, bag, vi...|
+--------------------+--------------------------+
only showing top 20 rows

In [0]:
bigram.transform(tokenized.select("DescOut")).show()

+--------------------+--------------------------+
|             DescOut|NGram_08853e7c5d02__output|
+--------------------+--------------------------+
|[rabbit, night, l...|      [rabbit night, ni...|
|[doughnut, lip, g...|      [doughnut lip, li...|
|[12, message, car...|      [12 message, mess...|
|[blue, harmonica,...|      [blue harmonica, ...|
|[gumball, coat, r...|      [gumball coat, co...|
|[skulls, , water,...|      [skulls ,  water,...|
|[feltcraft, girl,...|      [feltcraft girl, ...|
|[camouflage, led,...|      [camouflage led, ...|
|[white, skull, ho...|      [white skull, sku...|
|[english, rose, h...|      [english rose, ro...|
|[hot, water, bott...|      [hot water, water...|
|[scottie, dog, ho...|      [scottie dog, dog...|
|[rose, caravan, d...|      [rose caravan, ca...|
|[gingham, heart, ...|      [gingham heart, h...|
|[storage, tin, vi...|      [storage tin, tin...|
|[set, of, 4, knic...|      [set of, of 4, 4 ...|
|   [popcorn, holder]|          [popcorn holder]|


### Converting Words into Numerical Representations

In [0]:
# COMMAND ----------

from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer()\
  .setInputCol("DescOut")\
  .setOutputCol("countVec")\
  .setVocabSize(500)\
  .setMinTF(1)\
  .setMinDF(2)
    

In [0]:
fittedCV = cv.fit(tokenized)

In [0]:
fittedCV.transform(tokenized).show()

+--------------------+--------------------+--------------------+
 Description| DescOut| countVec|
+--------------------+--------------------+--------------------+
 RABBIT NIGHT LIGHT|[rabbit, night, l...|(500,[149,185,212...|
 DOUGHNUT LIP GLOSS |[doughnut, lip, g...|(500,[462,463,491...|
12 MESSAGE CARDS ...|[12, message, car...|(500,[35,41,166],...|
BLUE HARMONICA IN...|[blue, harmonica,...|(500,[10,16,36,35...|
 GUMBALL COAT RACK|[gumball, coat, r...|(500,[228,280,407...|
SKULLS WATER TRA...|[skulls, , water,...|(500,[11,40,133],...|
FELTCRAFT GIRL AM...|[feltcraft, girl,...|(500,[60,64,69],[...|
CAMOUFLAGE LED TORCH|[camouflage, led,...| (500,[264],[1.0])|
WHITE SKULL HOT W...|[white, skull, ho...|(500,[15,34,39,40...|
ENGLISH ROSE HOT ...|[english, rose, h...|(500,[34,39,40,46...|
HOT WATER BOTTLE ...|[hot, water, bott...|(500,[34,39,40,14...|
SCOTTIE DOG HOT W...|[scottie, dog, ho...|(500,[34,39,40,14...|
ROSE CARAVAN DOOR...|[rose, caravan, d...|(500,[46,297],[1....|
GINGHAM HEART DO...|[gingham, heart, ...|(500,[3,4,11,143,...|
STORAGE TIN VINTA...|[storage, tin, vi...|(500,[6,45,109,16...|
SET OF 4 KNICK KN...|[set, of, 4, knic...|(500,[0,1,49,70,3...|
 POPCORN HOLDER| [popcorn, holder]|(500,[21,296],[1....|
GROW A FLYTRAP OR...|[grow, a, flytrap...|(500,[36,45,378],...|
AIRLINE BAG VINTA...|[airline, bag, vi...|(500,[2,6,328],[1...|
AIRLINE BAG VINTA...|[airline, bag, vi...|(500,[0,2,6,328,4...|
+--------------------+--------------------+--------------------+
only showing top 20 rows

### Term frequency–inverse document frequency

In [0]:
# COMMAND ----------

tfIdfIn = tokenized\
  .where("array_contains(DescOut, 'red')")\
  .select("DescOut")\
  .limit(10)
tfIdfIn.show(10, False)

+---------------------------------------+
DescOut |
+---------------------------------------+
[gingham, heart, , doorstop, red] |
[red, floral, feltcraft, shoulder, bag]|
[alarm, clock, bakelike, red] |
[pin, cushion, babushka, red] |
[red, retrospot, mini, cases] |
[red, kitchen, scales] |
[gingham, heart, , doorstop, red] |
[large, red, babushka, notebook] |
[red, retrospot, oven, glove] |
[red, retrospot, plate] |
+---------------------------------------+

In [0]:
# COMMAND ----------

from pyspark.ml.feature import HashingTF, IDF
tf = HashingTF()\
  .setInputCol("DescOut")\
  .setOutputCol("TFOut")\
  .setNumFeatures(10000)
idf = IDF()\
  .setInputCol("TFOut")\
  .setOutputCol("IDFOut")\
  .setMinDocFreq(2)

In [0]:
# COMMAND ----------

idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).show(10, False)

+---------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
DescOut |TFOut |IDFOut |
+---------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
[gingham, heart, , doorstop, red] |(10000,[52,804,3372,6594,9808],[1.0,1.0,1.0,1.0,1.0])|(10000,[52,804,3372,6594,9808],[0.0,1.2992829841302609,1.2992829841302609,1.2992829841302609,1.2992829841302609])|
[red, floral, feltcraft, shoulder, bag]|(10000,[50,52,415,6756,8005],[1.0,1.0,1.0,1.0,1.0]) |(10000,[50,52,415,6756,8005],[0.0,0.0,0.0,0.0,0.0]) |
[alarm, clock, bakelike, red] |(10000,[52,4995,8737,9001],[1.0,1.0,1.0,1.0]) |(10000,[52,4995,8737,9001],[0.0,0.0,0.0,0.0]) |
[pin, cushion, babushka, red] |(10000,[52,610,2490,7153],[1.0,1.0,1.0,1.0]) |(10000,[52,610,2490,7153],[0.0,0.0,0.0,1.2992829841302609]) |
[red, retrospot, mini, cases] |(10000,[52,547,6703,8448],[1.0,1.0,1.0,1.0]) |(10000,[52,547,6703,8448],[0.0,0.0,0.0,1.0116009116784799]) |
[red, kitchen, scales] |(10000,[52,756,6452],[1.0,1.0,1.0]) |(10000,[52,756,6452],[0.0,0.0,0.0]) |
[gingham, heart, , doorstop, red] |(10000,[52,804,3372,6594,9808],[1.0,1.0,1.0,1.0,1.0])|(10000,[52,804,3372,6594,9808],[0.0,1.2992829841302609,1.2992829841302609,1.2992829841302609,1.2992829841302609])|
[large, red, babushka, notebook] |(10000,[52,2787,7022,7153],[1.0,1.0,1.0,1.0]) |(10000,[52,2787,7022,7153],[0.0,0.0,0.0,1.2992829841302609]) |
[red, retrospot, oven, glove] |(10000,[52,8242,8448,8667],[1.0,1.0,1.0,1.0]) |(10000,[52,8242,8448,8667],[0.0,0.0,1.0116009116784799,0.0]) |
[red, retrospot, plate] |(10000,[52,4925,8448],[1.0,1.0,1.0]) |(10000,[52,4925,8448],[0.0,0.0,1.0116009116784799]) |
+---------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+

### Word2Vec

Word2Vec is notable for capturing relationships between words based on their semantics. 
Here’s a simple example from the documentation:

In [0]:
# COMMAND ----------

from pyspark.ml.feature import Word2Vec
# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

In [0]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text",
  outputCol="result")

In [0]:
model = word2Vec.fit(documentDF)

In [0]:
result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [Hi, I, heard, about, Spark] => 
Vector: [0.012253652513027192,-0.06464210934937001,-0.007541720569133759]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.051534443129120124,0.025855228970093385,0.02741005045494863]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.0655867613852024,0.020301107689738276,-0.0846104308962822]

## Feature Manipulation

### PCA
Principal Components Analysis (PCA) is a mathematical technique for finding the most important
aspects of our data (the principal components). It changes the feature representation of our data by
creating a new set of features (“aspects”). Each new feature is a combination of the original features.

You’d want to use PCA if you have a large input dataset

In [0]:
# COMMAND ----------

from pyspark.ml.feature import PCA
pca = PCA().setInputCol("features").setK(2)
pca.fit(scaleDF).transform(scaleDF).show(20, False)

+---+--------------+------------------------------------------+
id |features |PCA_9e3929012a14__output |
+---+--------------+------------------------------------------+
0 |[1.0,0.1,-1.0]|[0.0713719499248417,-0.4526654888147805] |
1 |[2.0,1.1,1.0] |[-1.6804946984073723,1.2593401322219198] |
0 |[1.0,0.1,-1.0]|[0.0713719499248417,-0.4526654888147805] |
1 |[2.0,1.1,1.0] |[-1.6804946984073723,1.2593401322219198] |
1 |[3.0,10.1,3.0]|[-10.872398139848944,0.030962697060155975]|
+---+--------------+------------------------------------------+

## Interaction
The feature transformer Interaction allows you to create an
interaction between two variables manually. It just multiplies the two features together—something
that a typical linear model would not do for every possible pair of features in your data. This
transformer is currently only available directly in Scala but can be called from any language using the
RFormula. We recommend users just use RFormula instead of manually creating interactions.

### Polynomial Expansion
Polynomial expansion is used to generate interaction variables of all the input columns. With
polynomial expansion, we specify to what degree we would like to see various interactions. For
example, for a degree-2 polynomial, Spark takes every value in our feature vector, multiplies it by
every other value in the feature vector, and then stores the results as features. For instance, if we have
two input features, we’ll get four output features if we use a second degree polynomial (2x2). If we
have three input features, we’ll get nine output features (3x3). If we use a third-degree polynomial,
we’ll get 27 output features (3x3x3) and so on. This transformation is useful when you want to see
interactions between particular features but aren’t necessarily sure about which interactions to
consider.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import PolynomialExpansion
pe = PolynomialExpansion().setInputCol("features").setDegree(2)
pe.transform(scaleDF).show()

+---+--------------+----------------------------------------+
 id| features|PolynomialExpansion_7a7283793b02__output|
+---+--------------+----------------------------------------+
 0|[1.0,0.1,-1.0]| [1.0,1.0,0.1,0.1,...|
 1| [2.0,1.1,1.0]| [2.0,4.0,1.1,2.2,...|
 0|[1.0,0.1,-1.0]| [1.0,1.0,0.1,0.1,...|
 1| [2.0,1.1,1.0]| [2.0,4.0,1.1,2.2,...|
 1|[3.0,10.1,3.0]| [3.0,9.0,10.1,30....|
+---+--------------+----------------------------------------+

## Feature Selection
There are a number of ways to evaluate feature
importance once you’ve trained a model but another option is to do some rough filtering beforehand.
Spark has some simple options for doing that, such as ChiSqSelector.

### ChiSqSelector
ChiSqSelector leverages a statistical test to identify features that are not independent from the label
we are trying to predict, and drop the uncorrelated features. 

It’s often used with categorical data in
order to reduce the number of features you will input into your model, as well as to reduce the
dimensionality of text data (in the form of frequencies or counts). Since this method is based on the
Chi-Square test, there are several different ways we can pick the “best” features. The methods are
numTopFeatures, which is ordered by p-value; percentile, which takes a proportion of the input
features (instead of just the top N features); and fpr, which sets a cut off p-value.

In [0]:
# COMMAND ----------

from pyspark.ml.feature import ChiSqSelector, Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn\
  .transform(sales.select("Description", "CustomerId"))\
  .where("CustomerId IS NOT NULL")
prechi = fittedCV.transform(tokenized)\
  .where("CustomerId IS NOT NULL")
chisq = ChiSqSelector()\
  .setFeaturesCol("countVec")\
  .setLabelCol("CustomerId")\
  .setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi)\
  .drop("customerId", "Description", "DescOut").show()

+--------------------+----------------------------------+
 countVec|ChiSqSelector_101a32fbd9da__output|
+--------------------+----------------------------------+
(500,[149,185,212...| (2,[],[])|
(500,[462,463,491...| (2,[],[])|
(500,[35,41,166],...| (2,[],[])|
(500,[10,16,36,35...| (2,[],[])|
(500,[228,280,407...| (2,[],[])|
(500,[11,40,133],...| (2,[],[])|
(500,[60,64,69],[...| (2,[],[])|
 (500,[264],[1.0])| (2,[],[])|
(500,[15,34,39,40...| (2,[],[])|
(500,[34,39,40,46...| (2,[],[])|
(500,[34,39,40,14...| (2,[],[])|
(500,[34,39,40,14...| (2,[],[])|
(500,[46,297],[1....| (2,[],[])|
(500,[3,4,11,143,...| (2,[],[])|
(500,[6,45,109,16...| (2,[],[])|
(500,[0,1,49,70,3...| (2,[0,1],[1.0,1.0])|
(500,[21,296],[1....| (2,[],[])|
(500,[36,45,378],...| (2,[],[])|
(500,[2,6,328],[1...| (2,[],[])|
(500,[0,2,6,328,4...| (2,[0],[1.0])|
+--------------------+----------------------------------+
only showing top 20 rows